# This script reads metadata in a .csv, cleans and formats the data, and exports the result as a .csv for use in climate research.

In [ ]:
import os
from os.path import isfile
from os import stat, listdir
from datetime import datetime
import pandas as pd

## Fix version attribute

In [ ]:
def extract_version(row):
    if row.version[0] != "v":
        attributes = row.local_file.split("/")
        return attributes[-2]  # version is included as second to last item in local_file
    return row.version

df.version = df.apply(extract_version, axis=1)

## Correct file paths

In [ ]:
def update_path(row):
    path = row['local_file']
    if not isfile(path):
        if "/data2/synda/sdt/data" in path:
            path = path.replace("/sdt", "")
        elif "/data4/gsmwork/data/" in path:
            path = path.replace("gsmwork", "synda")
            if not isfile(path):
                path = path.replace("/output/", "/output/output/")
                path = path.replace("/output1/", "/output1/output1/")
                path = path.replace("/output2/", "/output2/output2/")
    if not isfile(path):
        raise Exception(f"Could not update path for: {row['local_file']}")
    return path

df['SCCASC_climatedata_path'] = df.apply(update_path, axis=1)

## Confirm all files exist on filesystem

In [ ]:
df['file_exists'] = df['SCCASC_climatedata_path'].apply(lambda x: isfile(x))
assert len(df[~df['file_exists']]) == 0

## Add OSCER Schooner paths

In [ ]:
def newpath(item):
    root = item.split("/")[1]
    remaining_path = "/".join(item.split("/")[2:])
    if root == "data":
        fullpath = f"/condo/climatedata3/{remaining_path}"
    else:
        fullpath = f"/condo/climate{root}/{remaining_path}"
    return fullpath

df['OSCER_schooner_path'] = df['SCCASC_climatedata_path'].apply(newpath)
df['local_file'] = df['SCCASC_climatedata_path']

## Segment out unique and duplicated entries

In [ ]:
unique = df[~df.duplicated(['filename'], False)]
duplicated = df[df.duplicated(['filename'], False)]
print(f"{len(unique)} unique files")
print(f"{len(duplicated)} duplicated files")

## Deduplicate

In [ ]:
mtime = lambda x: datetime.fromtimestamp(stat(x).st_mtime).isoformat()
issues = []
def max_version(grp):
    """ return single record from group """
    
    # max version based on string comparison
    records = grp[grp.version == grp.version.max()]
    
    if len(records) > 1:
        # most recent modified timestamp
        records['file_mtime'] = records.apply(lambda x: mtime(x.local_file), axis=1)
        records = records[records.file_mtime == records.file_mtime.max()]
    
    if len(records) > 1:
        # item with the most files in the same directory
        keep = None
        max_count = 0
        for record in records.to_dict(orient='records'):
            path, file = os.path.split(record['local_file'])
            file_count = len(listdir(path))
            if file_count > max_count:
                keep = record
                max_count = file_count
        records = pd.DataFrame([keep])
    
    if not len(records) == 1:
        issues.append(records.local_file)
    
    if issues:
        raise Exception("Could not determine which record to use for the following: ", issues)
    
    return records
    
deduplicated = duplicated.groupby(['filename'], as_index=False).apply(max_version)
print(f"{len(deduplicated)} deduplicated files")

## Merge unique and deduplicated

In [ ]:
print("Merging unique and deduplicated...")
cleaned = pd.concat([unique, deduplicated], ignore_index=True)

## Remove records that are after year 2101

In [ ]:
cleaned['beg_year'] = cleaned.time.str.split("-").apply(lambda x: int(x[0][0:4]))
cleaned = cleaned[cleaned.beg_year < 2101]

## Sort

In [ ]:
cleaned = cleaned.sort_values(['variable', 'model', 'experiment', 'time_frequency', 'filename'])

## Add human readable variable names and dimensions

In [ ]:
df_variable_names = pd.read_csv('variable_name_lookup_table.csv')
df_variable_dimensions = pd.read_csv('variable_dimension_lookup_table.csv')
df_tmp = pd.merge(cleaned, df_variable_names, on='variable')
df_complete = pd.merge(df_tmp, df_variable_dimensions, on='variable')

## Export selected columns to CSV

In [ ]:
headers = (
    'variable', 'variable_standard_name', 'variable_long_name', 'institute',
    'model', 'domain', 'dimensions', 'project', 'realm', 'ensemble',
    'experiment', 'time_frequency', 'time', 'version', 'filename', 'size',
    'OSCER_schooner_path', 'SCCASC_climatedata_path', 'checksum_type', 'checksum',
)
df_complete.to_csv("CMIP5_climatedata_full.csv", index=False, columns=headers)